In [1]:
from hepmc import *
import numpy as np

In [2]:
sarge = densities.Sarge(2, 3, 100.)

In [3]:
eeqqg = densities.ee_qq_ng(1, 100., 5., .3)

In [4]:
rambo_mapping = phase_space.RamboOnDiet(100., 3)
mapped = phase_space.MappedDensity(eeqqg, rambo_mapping)

In [5]:
%time sarge_sample = ImportanceMC(sarge)(eeqqg.pdf, 10000)

Event 1	(avg. trials per event: 9.000000)
Event 2	(avg. trials per event: 5.500000)	XS = 257.288428 pb +- ( 48.186992 pb = 18.728783 %)
Event 3	(avg. trials per event: 4.666667)	XS = 210.644131 pb +- ( 98.223028 pb = 46.629843 %)
Event 4	(avg. trials per event: 4.000000)	XS = 236.287103 pb +- ( 81.580717 pb = 34.526098 %)
Event 5	(avg. trials per event: 3.600000)	XS = 210.970884 pb +- ( 87.117369 pb = 41.293551 %)
Event 6	(avg. trials per event: 5.500000)	XS = 150.120194 pb +- ( 48.087610 pb = 32.032739 %)
Event 7	(avg. trials per event: 5.857143)	XS = 121.641793 pb +- ( 42.777138 pb = 35.166481 %)
Event 8	(avg. trials per event: 5.625000)	XS = 112.046649 pb +- ( 41.250880 pb = 36.815809 %)
Event 9	(avg. trials per event: 5.444444)	XS = 114.530752 pb +- ( 37.606508 pb = 32.835293 %)
Event 10	(avg. trials per event: 5.200000)	XS = 118.995140 pb +- ( 35.229479 pb = 29.605813 %)
Event 20	(avg. trials per event: 5.550000)	XS = 89.046091 pb +- ( 20.447539 pb = 22.962871 %)
Event 30	(avg. tr

In [6]:
densities.export_hepmc(100., sarge_sample, "../samples/qcd/2-3/sarge_weighted.hepmc")

In [7]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((10000, 12))
for i in range(10000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqqg.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(10000, 1./10000))

In [ ]:
densities.export_hepmc(100., sarge_sample_unweighted, "../samples/qcd/2-3/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(5)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
rambo = AcceptRejectSampler(mapped, max_y, 5)
%time rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
densities.export_hepmc(100., rambo_sample, "../samples/qcd/2-3/rambo.hepmc")

In [8]:
# just a local sampler (rambo)
met = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))
%time met_sample = met.sample(10000, np.random.rand(5))

Event 1	(batch acceptance rate: 0.000000)
Event 2	(batch acceptance rate: 0.000000)	max sequence length: 3
Event 3	(batch acceptance rate: 0.333333)	max sequence length: 3
Event 4	(batch acceptance rate: 0.250000)	max sequence length: 3
Event 5	(batch acceptance rate: 0.200000)	max sequence length: 3
Event 6	(batch acceptance rate: 0.166667)	max sequence length: 4
Event 7	(batch acceptance rate: 0.142857)	max sequence length: 5
Event 8	(batch acceptance rate: 0.125000)	max sequence length: 6
Event 9	(batch acceptance rate: 0.111111)	max sequence length: 7
Event 10	(batch acceptance rate: 0.100000)	max sequence length: 8
Event 20	(batch acceptance rate: 0.150000)	max sequence length: 11
Event 30	(batch acceptance rate: 0.100000)	max sequence length: 13
Event 40	(batch acceptance rate: 0.150000)	max sequence length: 18
Event 50	(batch acceptance rate: 0.160000)	max sequence length: 18
Event 60	(batch acceptance rate: 0.183333)	max sequence length: 18
Event 70	(batch acceptance rate: 0.17

In [9]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [10]:
# now sarge + local in MC3
importance = DefaultMetropolis(3 * 4, eeqqg, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(5, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
%time mc3_sample = update.sample(10000, np.random.rand(5))

/net/theorie/home/tjansse2/studium/masterarbeit/hep-monte-carlo/src/hepmc/core/markov/metropolis.py:47: RuntimeWarning: invalid value encountered in true_divide
  state.pdf / self.proposal_pdf(state, candidate))
/net/theorie/home/tjansse2/studium/masterarbeit/hep-monte-carlo/src/hepmc/core/markov/metropolis.py:47: RuntimeWarning: divide by zero encountered in true_divide
  state.pdf / self.proposal_pdf(state, candidate))


Event 1	(batch acceptance rate: 1.000000)
Event 2	(batch acceptance rate: 1.000000)	max sequence length: 1
Event 3	(batch acceptance rate: 1.000000)	max sequence length: 1
Event 4	(batch acceptance rate: 0.750000)	max sequence length: 2
Event 5	(batch acceptance rate: 0.800000)	max sequence length: 2
Event 6	(batch acceptance rate: 0.833333)	max sequence length: 2
Event 7	(batch acceptance rate: 0.857143)	max sequence length: 2
Event 8	(batch acceptance rate: 0.875000)	max sequence length: 2
Event 9	(batch acceptance rate: 0.888889)	max sequence length: 2
Event 10	(batch acceptance rate: 0.900000)	max sequence length: 2
Event 20	(batch acceptance rate: 0.800000)	max sequence length: 3
Event 30	(batch acceptance rate: 0.733333)	max sequence length: 3
Event 40	(batch acceptance rate: 0.650000)	max sequence length: 3
Event 50	(batch acceptance rate: 0.620000)	max sequence length: 3
Event 60	(batch acceptance rate: 0.616667)	max sequence length: 3
Event 70	(batch acceptance rate: 0.571429)

In [11]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
densities.export_hepmc(100., mc3_sample, "../samples/qcd/2-3/mc3.hepmc")